In [ ]:
import os, sys
#sys.path.append("/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages")

In [ ]:
from scipy.sparse import csr_matrix, eye, diags
from scipy.sparse.linalg import cg
from skimage.io import imread

import cv2
import matplotlib.pyplot as plt
import numpy as np

# Params

In [ ]:
PATH_TO_DATA = os.getcwd() + '/../data/test_set/mobius'
grid_params = {'sigma_spatial': 8, 
               'sigma_luma': 4, 
               'sigma_chroma': 4}
solver_params = {'max_iter_bisto': 10,
                 'max_iter_cg': 25,
                 'lambda': 128}

# Data

In [ ]:
def read_images(file_path, file_names):
    """Read images in a same folder
    
    Params:
        file_path (str): folder path
        file_names (list): list of file names
        
    Returns:
        imgs (list): list of images
    """
    
    imgs = [imread(os.path.join(file_path, name)) for name in file_names]
    
    print("Load %d images!" % len(imgs))

    return imgs

In [ ]:
def plot_images(imgs, names):
    """Plot images vertically
    
    Params:
        imgs(list): list of images
        names(list): list of figure titles
    """
    
    n = len(imgs)
    plt.figure(figsize = (20 * n, 20))
    
    for i in range(n):
        plt.subplot(n, 1, i + 1)
        plt.imshow(imgs[i])
        plt.title(names[i])

    plt.show()

# Color Convert

In [ ]:
# COPIED FROM https://github.com/poolio/bilateral_solver

RGB_TO_YUV = np.array([
    [ 0.299,     0.587,     0.114],
    [-0.168736, -0.331264,  0.5],
    [ 0.5,      -0.418688, -0.081312]])
YUV_TO_RGB = np.array([
    [1.0,  0.0,      1.402],
    [1.0, -0.34414, -0.71414],
    [1.0,  1.772,    0.0]])
YUV_OFFSET = np.array([0, 128.0, 128.0]).reshape(1, 1, -1)

def rgb2yuv(im):
    return np.tensordot(im, RGB_TO_YUV, ([2], [1])) + YUV_OFFSET

def yuv2rgb(im):
    return np.tensordot(im.astype(float) - YUV_OFFSET, YUV_TO_RGB, ([2], [1]))

# Bilateral Grid

In [ ]:
def build_W(yuv_im, param):
    """Create the grid coordinates and the blur matrix
    
    Params:
        yum_im (np.array): reference image in YUV color space
        param (dict): provide parameters
        
    Returns:
        S (np.array): of size num_voxels x num_verticals
        B_ (np.array): of size num_verticals x num_verticals
    
    """

    n_row, n_col, _ = yuv_im.shape
    nb_pixels = n_row * n_col
    
    # [1, 255, 255^2, 255^3, 255^4] -> [col, row, y, u, v]
    hashing_vector = np.zeros(5)
    for i in range(5):
        hashing_vector[i] = 255 ** i

    # a dict from hash number to index
    hashed_coords = {}
    
    vertical_idx = 0
    pixel_idx = 0
    splat_idxs = np.zeros(nb_pixels)
    
    for col_idx in range(n_col):
        for row_idx in range(n_row):
            coord = np.zeros(5)
            coord[0] = row_idx // param['sigma_spatial']
            coord[1] = col_idx // param['sigma_spatial']
            coord[2] = yuv_im[row_idx, col_idx, 0] // param['sigma_luma']
            coord[3] = yuv_im[row_idx, col_idx, 1] // param['sigma_chroma']
            coord[4] = yuv_im[row_idx, col_idx, 2] // param['sigma_chroma']
            
            hashed_coord = sum([coord[i] * hashing_vector[i] for i in range(5)])
            
            if hashed_coord in hashed_coords:
                splat_idxs[pixel_idx] = hashed_coords[hashed_coord]
            else:
                hashed_coords[hashed_coord] = vertical_idx
                splat_idxs[pixel_idx] = vertical_idx
                vertical_idx += 1
            
            pixel_idx += 1
            
    n_vertices = len(hashed_coords)
    
    # (data, (row_idx, col_idx))
    S = csr_matrix((np.ones(nb_pixels), (splat_idxs.astype(int), np.arange(nb_pixels))))
    B_ = eye(n_vertices) * 10 # 2 * 5
    
    func_mask = np.vectorize(lambda x: x in hashed_coords)
    func_coords = np.vectorize(lambda x: hashed_coords[x])
    
    # Convolution [1, 2, 1] ^ 5
    for offset in (-1, 1):
        for i in range(5):
            offset_hashed_coord = offset * hashing_vector[i]
            centers = np.array(list(hashed_coords.keys()))
            neighbors = centers + offset_hashed_coord
            mask = func_mask(neighbors)

            coords_centers = func_coords(centers[mask])
            coords_neighbors = func_coords(neighbors[mask])

            nb_neighbors = len(coords_centers)
            B_temp = csr_matrix((np.ones(nb_neighbors), (coords_centers, coords_neighbors)), shape = (n_vertices, n_vertices))
            B_ += B_temp
            
    return S, B_

In [ ]:
def init_solution(S, confidence, target):
    """Naive solution by performing convolution
    
    Params:
        S: matrix of grid_coordinates
        confidence: confidence map
        target: target map
        
    Returns:
        x_hat: solution in coordinate
        y_hat: solution in bilateral space
    """

    s_ct = S.dot(confidence * target)
    s_c = S.dot(confidence)
    
    y_hat = s_ct / s_c
    x_hat = S.T.dot(y_hat).reshape(confidence.shape[::-1]).T
    
    return x_hat, y_hat

# Bilateral Solver

In [ ]:
def bistochastization(S, B_, max_iter = 10):
    """Make all rows and columns have the same mean value
    
    Params:
        S: grid_coordinates
        B_: blur matrix
        max_iter: number of iterations
        
    Returns:
        Dn (csr matrix)
        Dm (csr matrix) 
    """
    
    # S.sum(1)
    m = S.dot(np.ones(S.shape[1]))
    n = np.ones(S.shape[0])
    
    for _ in range(max_iter):
        num = n * m
        den = B_.dot(n)
        n = np.sqrt(np.divide(num, den))
    
    # Correct m to satisfy the assumption of bistochastization regardless
    # of how many iterations have been run.
    m = n * (B_.dot(n))

    return diags(n, 0, format = 'csr'), diags(m, 0, format = 'csr')

In [ ]:
def build_A(S, B_, D_n, D_m, conf, lamb):
    """Calculate A
    
    Params:
        S: grid_coordinates
        B_: blur matrix
        Dn: bistochastic matrix
        Dm: bistochastic matrix
        conf: confidence map
        lamb: coefficient for bilateral term
        
    Returns:
        A: of size num_voxels x num_voxels  
    """

    A = lamb * (D_m - D_n.dot(B_.dot(D_n))) + diags(np.squeeze(S.dot(conf.reshape((-1, 1)))), format = 'csr')
    
    return A

In [ ]:
def build_b(S, conf, target):
    """Calculate b
    
    Params:
        S: grid_coordinates
        conf: confidence map
        target: target map
        
    Returns:
        b: of size num_voxels x 1
    """
    
    dot_prod = (conf * target).reshape((-1, 1))
    
    return S.dot(dot_prod)

In [ ]:
def naive_preconditioner(A):
    """Jacobi preconditioner for A
    
    Params:
        A: of size num_voxels x num_voxels
        
    Returns:
        M_inv: jacobi preconditioner
    """
    
    M_inv = diags(1. / A.diagonal(), 0)
    
    return M_inv

In [ ]:
def cg_solver(A, b, M_inv, y0, max_iter = 50):
    """Solve Ay = b
    
    Params:
        A: of size num_voxels x num_voxels
        b: of size num_voxels x 1
        M_inv: preconditioner for A
        y0: initial solution
        max_iter: maximum number of iterations
        
    Return:    
        y: solution
    """
    
    y = cg(A, b, M = M_inv, maxiter = max_iter, x0 = y0)
    
    return y

# Oracle

In [ ]:
def bilateral_grid(ref, scaled_confidence, scaled_target, grid_params):
    """Oracle for bilateral grid
    
    Params:
        ref: reference image in RGB color space
        scaled_confidence: scaled confidence map
        scaled_target: scaled target map
        grid_params: parameters for the oracle
        
    Returns:
        S: grid coordinate
        B_: blur matrix
        x0: naive solution in coordinate space
        y0: naive solution in bilateral space
    """
    
    print("Convert image to YUV space...")
    ref_yuv = rgb2yuv(ref)
    print("Image size: ", ref_yuv.shape)
    print("Min pixel value: ", ref_yuv.min((0, 1)))
    print("Max pixel value: ", ref_yuv.max((0, 1)))
    
    print("\nBuild S and B...")
    S, B_ = build_W(ref_yuv, grid_params)
    
    print("\nCalculate solution...")
    x0, y0 = init_solution(S, scaled_confidence, scaled_target)
    print("\nFinish!")
    
    results = {'S': S, 'B_': B_, 'x0': x0, 'y0': y0}
    
    return results

In [ ]:
def bilateral_solver(scaled_confidence, scaled_target, grid_results, img_shape, solver_params):
    """Oracle for bilateral solver
    
    Params:
        scaled_confidence: scaled confidence map
        scaled_target: scaled target map
        grid_results: solutions from bilateral grid
        img_shape: shape of the output map
        solver_params: parameters for the oracle
    
    Returns:
        x: final solution in coordinate space
    """
    
    S, B_, y0 = grid_results['S'], grid_results['B_'], grid_results['y0']
    
    print("Bistochastization S and B...")
    D_n, D_m = bistochastization(S, B_, solver_params['max_iter_bisto'])
    
    print("\nCalculate A and b...")
    A = build_A(S, B_, D_n, D_m, scaled_confidence, solver_params["lambda"])
    b = b = build_b(S, scaled_confidence, scaled_target)
    
    print("\nPreconditionning...")
    M_inv = naive_preconditioner(A)
    
    print("\nSolve Ay = b...")
    y, flag = cg_solver(A, b, M_inv, y0, solver_params['max_iter_cg'])
    
    assert(flag >= 0), "Conjugate gradient solve failed!"
    
    print("\nCalculate x...")
    x = S.T.dot(y).reshape(img_shape).T
    print("\nFinish!")
    
    return x

# Experiments

In [ ]:
reference, target, confidence = read_images(PATH_TO_DATA, ['reference.png', 'target.png', 'confidence.png'])
print("Image Size:", reference.shape)

In [ ]:
plot_images([reference, target, confidence], 
            ['reference', 'target', 'confidence'])

In [ ]:
scaled_t = target.T.reshape(-1, 1).astype(np.double) / (pow(2, 16) - 1)
scaled_c = confidence.T.reshape(-1, 1).astype(np.double) / (pow(2, 16) - 1)

In [ ]:
grid_results = bilateral_grid(reference, scaled_c, scaled_t, grid_params)

In [ ]:
out_shape = reference.shape[:2][::-1]
x = bilateral_solver(scaled_c, scaled_t, grid_results, out_shape, solver_params)

In [ ]:
plot_images([target, grid_results['x0'].reshape(x.shape), x], 
            ['input', 'grid output', 'solver output'])

# Stéréo - Test

In [ ]:
def gen_output_from_param(path_to_data, image_names, grid_params, solver_params):
    reference, target, confidence, ground_truth = read_images(path_to_data, image_names)
    
    scaled_t = target.T.reshape(-1, 1).astype(np.double) / (pow(2, 16) - 1)
    scaled_c = confidence.T.reshape(-1, 1).astype(np.double) / (pow(2, 16) - 1)
    
    grid_results = bilateral_grid(reference, scaled_c, scaled_t, grid_params)
    out_shape = reference.shape[:2][::-1]
    
    x = bilateral_solver(scaled_c, scaled_t, grid_results, out_shape, solver_params)
    
    value = np.linalg.norm(ground_truth - x)
    
    return x, value

In [ ]:
def test_stereo(path_to_data, data_sets, image_names, grid_range, solver_range):
    
    spatial_values = []
    luma_values = []
    chroma_values = []
    lambda_values = []
    
    for data_set in data_sets:
        i = 0
        max_i = np.sum(np.array([len(grid_range_value[1]) 
                                 for grid_range_value in grid_range.values()])) \
                + len(solver_range['lambda'][1])
        
        path_to_images = os.path.join(PATH_TO_DATA, data_set)
        # Check spatial influence :
        grid_params = {
            'sigma_luma': grid_range['sigma_luma'][0],
            'sigma_chroma': grid_range['sigma_chroma'][0],
                     }
        solver_params = {
            'max_iter_bisto': solver_range['max_iter_bisto'],
            'max_iter_cg': solver_range['max_iter_cg'],
            'lambda': solver_range['lambda'][0]
        }
        for sigma_spatial in grid_range['sigma_spatial'][1]:
            print('Dataset : {}, step {}/{}.'.format(data_set, i, max_i))
            i += 1
            grid_params['sigma_spatial'] = sigma_spatial
            x, value = gen_output_from_param(path_to_images, image_names, grid_params, solver_params)
            spatial_values.append((x, value))
            
        # Check luma influence :
        grid_params = {
            'sigma_spatial': grid_range['sigma_spatial'][0],
            'sigma_chroma': grid_range['sigma_chroma'][0],
                     }
        solver_params = {
            'max_iter_bisto': solver_range['max_iter_bisto'],
            'max_iter_cg': solver_range['max_iter_cg'],
            'lambda': solver_range['lambda'][0]
        }
        for sigma_luma in grid_range['sigma_luma'][1]:
            print('Dataset : {}, step {}/{}.'.format(data_set, i, max_i))
            i += 1
            grid_params['sigma_luma'] = sigma_luma
            _, value = gen_output_from_param(path_to_images, image_names, grid_params, solver_params)
            luma_values.append((x, value))
            
        # Check chroma influence :
        grid_params = {
            'sigma_spatial': grid_range['sigma_spatial'][0],
            'sigma_luma': grid_range['sigma_luma'][0],
                     }
        solver_params = {
            'max_iter_bisto': solver_range['max_iter_bisto'],
            'max_iter_cg': solver_range['max_iter_cg'],
            'lambda': solver_range['lambda'][0]
        }
        for sigma_chroma in grid_range['sigma_chroma'][1]:
            print('Dataset : {}, step {}/{}.'.format(data_set, i, max_i))
            i += 1
            grid_params['sigma_chroma'] = sigma_chroma
            _, value = gen_output_from_param(path_to_images, image_names, grid_params, solver_params)
            chroma_values.append((x, value))
            
        
        # Check lambda influence :
        grid_params = {
            'sigma_spatial': grid_range['sigma_spatial'][0],
            'sigma_luma': grid_range['sigma_luma'][0],
            'sigma_chroma': grid_range['sigma_chroma'][0],
                     }
        solver_params = {
            'max_iter_bisto': solver_range['max_iter_bisto'],
            'max_iter_cg': solver_range['max_iter_cg'],
        }
        for lamb in solver_range['lambda'][1]:
            print('Dataset : {}, step {}/{}.'.format(data_set, i, max_i))
            i += 1
            solver_params['lambda'] = lamb
            _, value = gen_output_from_param(path_to_images, image_names, grid_params, solver_params)
            lambda_values.append((x, value))
        
    return spatial_values, luma_values, chroma_values, lambda_values
   

In [ ]:
PATH_TO_DATA = os.getcwd() + '/../data/test_set/'
data_sets = ['art', 'books', 'mobius']
image_names = ['reference.png', 'target.png', 'confidence.png', 'gt.png']
grid_range = {
    'sigma_spatial': (8, [1]),# 4, 8, 16, 32])
    'sigma_luma': (4, [0.5]),#, 1, 2, 4, 8, 16]) 
    'sigma_chroma': (4, [0.5]),#, 1, 2, 4, 8, 16])
}
solver_range = {
    'max_iter_bisto': 10,
    'max_iter_cg': 25,
    'lambda': (128, [64]),#, 128, 256]
}

In [ ]:
test_output = test_stereo(PATH_TO_DATA, data_sets, image_names, grid_range, solver_range)